In [1]:
import cv2
import os
import time
import numpy as np
from scipy import ndimage, signal
from matplotlib import pyplot as plt
import shutil
import pickle

In [6]:
kSC = np.array([[-1 / 8, -1 / 8, -1 / 8],  # from class material
                [-1 / 8, 1, -1 / 8],
                [-1 / 8, -1 / 8, -1 / 8]])

def convert(img, target_type_min, target_type_max, target_type):
    imin = img.min()
    imax = img.max()

    a = (target_type_max - target_type_min) / (imax - imin)
    b = target_type_max - a * imax
    new_img = (a * img + b).astype(target_type)
    return new_img
def toGray(im, im_L=10, im_W=10, contrast=1.0):
    # normalize
    gray = np.zeros([im_L, im_W])
    for j in range(im_L):
        for i in range(im_W):
            gray[j][i] = max(im[j][i]) * contrast
    gray = convert(gray, 0, 255, np.uint8)
    return gray
def threshold(frame):
    # for j in range(len(frame)):
    #     for i in range(len(frame[0])):
    #         print(frame[j][i])
    #
    # print("---------------------------------")
    out = np.full_like(frame, 128)
    out[frame < -4] = 0
    # out[frame > 4] = 255
    return out
def SpacialContrast(imGray):
    conv_gray = signal.convolve2d(imGray, kSC, 'same')
    # print(np.int32(conv_gray))
    fSCgray = np.array(threshold(conv_gray), dtype=np.uint8)
    return fSCgray

In [7]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

In [8]:
# Generate Sequence String for Each Sample 0-200
dirSamples = "Sample List Gray"
dirGroups = "ShapeGroups GRAY"
dirSequences = "Sequences/"
idx = -1
numSamples = 200
for sample_idx in range(numSamples):
    samplename = "sample" + str(sample_idx)
    samplefile = "Sample List Gray/sample" + str(sample_idx)
    sequence = ''
    for file in os.listdir(samplefile):
        # print('\t' + file)
        getpath = find(file, dirGroups)
        if type(getpath) != type(None):
            # print('\t' + getpath)
            sequence += getpath[22]
        else:
            sequence += '-'
    print(samplename, sequence)
    with open(dirSequences + samplename, 'w') as output:
        output.write(sequence)
    idx += 1

sample0
01-243-01
sample1
20-021-20
sample2
222-4-033
sample3
010242-00
sample4
514-3-031
sample5
-21-20-05
sample6
100331--0
sample7
0-3-21-53
sample8
303530-12
sample9
2411-3---
sample10
300-5231-
sample11
025042101
sample12
-1-4-5203
sample13
5-01145--
sample14
000435133
sample15
-24551413
sample16
-102551--
sample17
5-333-44-
sample18
12242-115
sample19
-45-51400
sample20
2-00-2135
sample21
11-20053-
sample22
0103504--
sample23
521125304
sample24
324352-31
sample25
-11512-21
sample26
333-5-41-
sample27
14111231-
sample28
-53141245
sample29
150-25122
sample30
020--552-
sample31
14425-01-
sample32
0-2452540
sample33
032052142
sample34
3-415--2-
sample35
431403-31
sample36
251--2510
sample37
101-5213-
sample38
404340--4
sample39
2-03-4103
sample40
13-45024-
sample41
3--50311-
sample42
-35305542
sample43
055513335
sample44
0-2--2552
sample45
41-1054-0
sample46
00-2530--
sample47
15--0454-
sample48
124104005
sample49
3312110-3
sample50
342510-03
sample51
4035-4-00
sample52
453-140-2
sam

In [3]:
# Find Num of Occurances for a Specific Sequence
def findNumSequence(sequenceToFind):
    dirSequences = "Sequences/"
    count = 0
    for files in os.listdir(dirSequences):
        with open(dirSequences + files, 'r') as file:
            sequence = file.read()
        if sequenceToFind in sequence:
            # print(sequence)
            count += 1
    return count
# prints samples where sequence is given
def findSequences(sequenceToFind):
    dirSequences = "Sequences/"
    samples = []
    for files in os.listdir(dirSequences):
        with open(dirSequences + files, 'r') as file:
            sequence = file.read()
        if sequenceToFind in sequence:
            samples.append(int(files[6:]))
            # print(files)
            # print(sequence)
    return samples

In [56]:
# Find most frequent 3 shape sequence
# print(findNumSequence('011'))
allsequences = np.zeros([6,6,6])
for first in range(6):
    for second in range(6):
        for third in range(6):
            seq = str(first) + str(second) + str(third)
            allsequences[first][second][third] = findNumSequence(seq)
allsequences[1,4,2] = 0
allsequences[0,5,3] = 0
allsequences[1,0,1] = 0
allsequences[3,5,2] = 0
allsequences[5,3,0] = 0
allsequences[0,2,5] = 0
maxidx_flat = np.argmax(allsequences)
print(np.unravel_index(maxidx_flat, [6,6,6]))
print(findNumSequence('530'))

(0, 4, 3)
8


In [8]:
trues = findSequences('025')
print(sorted(trues), len(trues))
y_set = np.zeros(200)
for true_idx in trues:
    y_set[true_idx] = 1
# print(y_set)

[11, 16, 60, 71, 72, 92, 185] 7
